In [9]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import supervision as sv

In [21]:
def load_yolo_labels(label_path) -> np.array:
    labels = []
    with open(label_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            cls, x_center, y_center, width, height = map(float, parts)
            labels.append([cls, x_center, y_center, width, height])
    return np.array(labels)

def yolo_to_detections(yolo_labels, image_shape):
    height, width = image_shape[:2]
    
    # Converti le coordinate da normalizzate a pixel
    boxes = []
    classes = []
    for label in yolo_labels:
        cls, x_center, y_center, w, h = label
        x_center *= width
        y_center *= height
        w *= width
        h *= height

        # Converti da (x_center, y_center, w, h) a (x_min, y_min, x_max, y_max)
        x_min = x_center - w / 2
        y_min = y_center - h / 2
        x_max = x_center + w / 2
        y_max = y_center + h / 2

        boxes.append([x_min, y_min, x_max, y_max])
        classes.append(int(cls))
    
    # Converti in array NumPy
    boxes = np.array(boxes)
    classes = np.array(classes)

    return sv.Detections(xyxy=boxes, class_id=classes)


In [11]:
BASE = "C:\\Users\\marco\\OneDrive\\Desktop\\my-dataset"

IMGS = os.path.join(BASE, "all_images")
LABELS = os.path.join(BASE, "all_labels")

OUT_DIR = 'C:\\Users\\marco\\OneDrive\\Documents\\GitHub\\Basketball_Video_Analysis\\ball-detection\\images_annotated'

In [12]:
images_files = [os.path.join(IMGS, file) for file in os.listdir(IMGS) if file.endswith(".jpg")]
labels_files = [os.path.join(LABELS, file) for file in os.listdir(LABELS) if file.endswith(".txt")]

In [13]:
len(labels_files), len(images_files)

(3168, 3483)

In [14]:
class_names = ['rim', 'ball']

In [51]:
def show_detections(class_names=['rim', 'ball'], rows=5, cols=5):
    class_ids = [class_names.index(name) for name in class_names]

    ball_images = []
    titles = []

    for image_path, label_path in zip(images_files, labels_files):
        img = cv2.imread(image_path)

        labels = load_yolo_labels(label_path)
        labels = np.array(labels, dtype=np.float32)
        mask = np.isin(labels[:, 0], class_ids)
        labels = labels[mask]
        if labels.size == 0:
            continue
        a = np.random.randint(4)
        if a == 0:
            continue
        try:
            detections = yolo_to_detections(labels, img.shape)
        except Exception as e:
            print("Labels non trovate per", image_path)
        bbox = detections.xyxy[0]
        ball = img[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
        ball_images.append(ball)
        titles.append(class_names[int(detections.class_id[0])])
        if len(ball_images) == rows * cols:
            break

    sv.plot_images_grid(ball_images, (rows, cols), titles=titles
    , size=(20, 20))


In [ ]:
show_detections()